In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [4]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

alpha = 0.9
#make beta a list of values
betas = [0.1, 0.2, 0.5, 0.7, 0.9, 0.95, 0.99, 0.999]
objectiv_values = []
exp_profits = []
CVaR_values = []

Any[]

In [5]:
for beta in betas
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)
    unregister(model, :p_DA)

    # Define the decision variables for hour
    @variable(model, p_DA[1:hours])
    @variable(model, delta[1:W,1:hours])
    @variable(model, delta_up[1:W,1:hours])
    @variable(model, delta_down[1:W,1:hours])
    @variable(model, zeta)
    @variable(model, eta[1:W] >= 0)

    # Define the objective function
    @objective(model, Max, (1-beta) *  sum(1/W*(scenarios[i][hour,"price"] * p_DA[hour]
        + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
        + beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W))) for i in 1:W, hour in 1:hours))

    # Define the constraints
    @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
    @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)

    @constraint(model, [i in 1:W], -1 * sum(scenarios[i][hour,"price"] * p_DA[hour]
    + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
    - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) for hour in 1:hours) + zeta - eta[i] <= 0)



    # Solve the optimization problem
    optimize!(model)

    exp_profit = sum(1/W.*(scenarios[i][hour,"price"] * p_DA[hour]
    + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
    - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) for i in 1:W, hour in 1:hours))

    CVaR = (value.(zeta) - (1/(1-alpha)) * sum(1/W .* value.(eta[i]) for i in 1:W))

    #println(model)

    # Print the termination status
    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Optimal solution found")
        println("p_DA: ", value.(p_DA))
        
        push!(objectiv_values, objective_value(model))
        push!(exp_profits, exp_profit)
        push!(CVaR_values, CVaR)
    else
        println("No optimal solution found")
    end
end

Optimal solution found
p_DA: [22.33724183870968, 14.389693451612905, 15.949935451612903, 13.469580564516129, 9.242903161290323, 7.841935483870968, 145.9269354516129, 2.7339112741935483, 1.1038967741935486, 146.94499998387096, 0.7919354838709678, 0.7419306451612904, 142.91516125806453, 22.92266117741936, 135.9887096612903, 22.37141930645161, 146.0248386935484, 149.60145158064518, 5.493548387096774, 152.74596772580645, 4.493516129032258, 16.500016064516128, 6.808774177419355, 149.57870966129033]
Optimal solution found
p_DA: [22.33724183870968, 14.389693451612905, 15.949935451612903, 13.469580564516129, 9.242903161290323, 7.841935483870968, 145.9269354516129, 2.7339112741935483, 1.1038967741935486, 146.94499998387096, 0.7919354838709678, 0.7419306451612904, 142.91516125806453, 17.20006443548387, 135.9887096612903, 19.107161209677415, 146.0248386935484, 149.60145158064518, 5.493548387096774, 152.74596772580645, 4.493516129032258, 14.784612903225804, 6.808774177419355, 149.57870966129033]
O